# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Нас интересуют такие вопросы:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

В самом начале импортируем необходимые библиотеки.

In [1]:
# импорт библиотеки pandas
import pandas as pd
# импорт метода Display
from IPython.display import display
# импорт библиотеки pymystem3 для лемматизации
from pymystem3 import Mystem
m = Mystem()
# вызов контейнера Counter
from collections import Counter

Прочитаем файл с данными и сохраним его в переменной `df`.

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('data.csv')

Посмотрим, какие данные у нас имеются.

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в нашей таблице двенадцать столбцов. Типы данных, встречаемых в столбцах: `object`, `int` и `float`.

Описание данных согласно документации:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы — данные о клиенте банка. Колонки рассказывют нам о том, какой у клиента опыт работы, какое у него образование, семейный статус. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. В названиях колонок нет проблем со стилем, но встречаются пропуски в данных, поэтому нужно устранить проблемы в данных, чтобы двигаться дальше.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала посчитаем, сколько в таблице пропущенных значений.

In [5]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущенные значения влияют на исследование. Пропуски в `days_employed` не важны для проверки наших гипотез. Достаточно заменить их явными обозначениями.

Но пропуски в `total_income` могут помешать поиску зависимости между уровнем дохода и возврата кредита в срок. Попробуем понять причину их появления и устранить их.

In [6]:
#заменим пропуски в столбце `days_employed` на 0
df['days_employed'] = df['days_employed'].fillna(0)

Далее заменим пропуски в стобце `total_income` медианными значениями по каждому из типов занятости.

In [7]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

Проверим, что больше нет неизвестных значений.

In [8]:
# попробуем найти строки, в которых мы заполняли пропуски
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Заполнили пропущенные значения, которые могли появиться из-за ошибки при сборе данных или прочих технических сбоев.

### Замена типа данных

Заменим тип данных float в столбцах `days_employed` и `total_income`. Так как исходное значение у нас `float`, то воспользуемся для преобразования в `int` методом `astype()`, метод `to_numeric()` нам тут не подойдёт.

In [9]:
# преобразование значений столбцов в нужный тип
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Проверим преобразование.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Как мы видим, у нас остались только столбцы типов `int` и `object`.

### Обработка дубликатов

Посчитаем количество явных дубликатов. Сначала приведём все значения с типом `object` к единому нижнему регистру.

In [11]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

In [12]:
# подсчёт явных дубликатов
print("Кол-во дубликатов: {}".format(df.duplicated().sum()))

Кол-во дубликатов: 71


Удалим явные дубликаты методом `drop_duplicates()`

In [13]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Снова перерсчитаем явные дубликаты, чтобы убедиться, что мы от них избавились.

In [14]:
# подсчёт явных дубликатов
print("Кол-во дубликатов: {}".format(df.duplicated().sum()))

Кол-во дубликатов: 0


**Вывод**

Обнаружили явные дубликаты и почистили от них наши данные. Явные дубликаты могли появиться из-за ошибки при сборе данных или прочих технических сбоев. Ручной поиск дубликатов не проводился, так как нет столбца, по которому можно идентифицировать неявные дубликаты.

### Лемматизация

Проведём лемматизацию столбца с целями получения кредита. Для этого выделим значения из столбца `purpose`.

In [15]:
# Лемматизируем с использованием метода `join()`
lemmas = m.lemmatize(' '.join(df['purpose']))
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


**Вывод**

Самые популярные слова:
* недвижимость;
* жилье;
* автомобиль;
* операция;
* свадьба;
* строительство.

### Категоризация данных

Для того, чтобы нам ответить на некоторые из вопросов исследования, категоризация некоторых данных. Добавим два новых столбца, один из которых будет отражать категорию цели получения кредита, второй будет отражать диапазон ежемесячного дохода клиента. 

In [16]:
# категоризация уровня дохода и добавление нового столбца
max_income = df['total_income'].max()
ranges = [0, 50000, 100000, 150000, 200000, 250000, 300000, max_income]
labels_for_ranges = ['от 0 до 50000', 'от 50000 до 100000', 'от 100000 до 150000', 'от 150000 до 200000',
          'от 200000 до 250000', 'от 250000 до 300000', 'от 300000 и более']
df['income_category'] = pd.cut(df['total_income'], bins=ranges, labels=labels_for_ranges)

Для категоризации цели кредита сделаем функцию, которая будет работать с лемматизированными словами из столбца, потому что так риск ошибиться меньше.

In [17]:
# функция категоризации цели кредита
def purpose_categorize(purpose):
    purpose = m.lemmatize(purpose)
    if ('недвижимость' in purpose or
    'жилье' in purpose or
    'строительство' in purpose or
    'коммерческий' in purpose or
    'ремонт' in purpose):
        return 'недвижимость'
    elif 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'образование' in purpose:
        return 'образование'
    elif 'свадьба' in purpose:
        return 'свадьба'
    else:
        return 'разное'

In [18]:
# добавим новые столбцы в нашу таблицу
df['purpose_category'] = df['purpose'].apply(purpose_categorize)

In [19]:
# поглядим, что у нас получилось
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,от 250000 до 300000,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,от 100000 до 150000,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,от 100000 до 150000,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,от 250000 до 300000,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,от 150000 до 200000,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем,от 200000 до 250000,недвижимость
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем,от 150000 до 200000,автомобиль
21451,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,от 50000 до 100000,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,от 200000 до 250000,автомобиль


Для формирования выводов нам не понадобятся все колонки, поэтому оставим только необходимые.

In [20]:
# сформируем копию таблицы с нужными нам столбцами и проверим
df_research = df[['children', 'family_status', 'debt', 'income_category', 'purpose_category']]
display(df_research)

,children,family_status,debt,income_category,purpose_category
0,1,женат / замужем,0,от 250000 до 300000,недвижимость
1,1,женат / замужем,0,от 100000 до 150000,автомобиль
2,0,женат / замужем,0,от 100000 до 150000,недвижимость
3,3,женат / замужем,0,от 250000 до 300000,образование
4,0,гражданский брак,0,от 150000 до 200000,свадьба
...,...,...,...,...,...
21449,1,гражданский брак,0,от 200000 до 250000,недвижимость
21450,0,женат / замужем,0,от 150000 до 200000,автомобиль
21451,1,гражданский брак,1,от 50000 до 100000,недвижимость
21452,3,женат / замужем,1,от 200000 до 250000,автомобиль


**Вывод**

Выделили интересующие нас данные для дальнейшего исследования. Были заполнены пропуски в данных, устранены дубликаты и проведена категоризация данных. Все эти шаги повысят точность нашего исследования и облегчат поиски ответов на вопросы исследования.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим, сколько вообще детей указано у клиентов в нашей таблице. Для этого сгруппируем данные по столбцу `children` и посчитаем количество клиентов методом `count()`.

In [21]:
# группируем и считаем
display(df_research.groupby('children')['debt'].count())

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

Видим аномальные значения. `-1` ребёнка быть не может, поэтому предположим, что это неправильная запись числа `1`. `20` детей тоже маловероятно, поэтому предположим, что неправильная запись числа `2`. Исправим наши данные.

In [22]:
# заменяем аномальные значения на скорректированные и смотрим, что получилось
df_research.loc[:, 'children'] = df_research['children'].replace(-1, 1)
df_research.loc[:, 'children'] = df_research['children'].replace(20, 2)
# группируем и считаем
display(df_research.groupby('children')['debt'].count())

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

Далее посчитаем количество клиентов, которые имели задолженность по кредитам с группировкой по количеству детей. Также посмотрим на процентное соотношение внутри каждой категории.

In [23]:
# задаём процентное форматирование, группируем и выводим результат
pd.set_option('display.float_format', '{:.2%}'.format)
df_research.groupby('children')[['debt']].agg(['count', 'sum', 'mean'])

debt            
          count   sum  mean
children                   
0         14091  1063 7.54%
1          4855   445 9.17%
2          2128   202 9.49%
3           330    27 8.18%
4            41     4 9.76%
5             9     0 0.00%

**Вывод**

Клиенты без детей оказались чуть дисциплинированнее. Видим, что количество детей не оказывает влияние на возврат кредита в срок. В каждой категории схожий процент просрочек. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для начала сгруппируем клиентов по семейному положению и посмотрим, какие категории у нас есть.

In [24]:
# группируем и считаем
display(df_research.groupby('family_status')['debt'].count())

family_status
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
не женат / не замужем     2810
Name: debt, dtype: int64

Далее посчитаем количество клиентов, которые имели задолженность по кредитам с группировкой по семейному положению. Для наглядности картины также нужно рассмотреть процентное соотношение должников к общему количеству клиентов в каждой категории.

In [25]:
# группируем и выводим результат
df_research.groupby('family_status')[['debt']].agg(['count', 'sum', 'mean'])

debt           
                       count  sum  mean
family_status                          
в разводе               1195   85 7.11%
вдовец / вдова           959   63 6.57%
гражданский брак        4151  388 9.35%
женат / замужем        12339  931 7.55%
не женат / не замужем   2810  274 9.75%

**Вывод**

Результаты показали, что вдовец / вдова реже всего допускают задолженность, а клиенты, которые не состоят в официальном браке - наименее дисциплинированные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сначала сгруппируем клиентов по категориям уровня доходов.

In [26]:
# группируем и считаем
display(df_research.groupby('income_category')['debt'].count())

income_category
от 0 до 50000           372
от 50000 до 100000     4091
от 100000 до 150000    7160
от 150000 до 200000    4764
от 200000 до 250000    2254
от 250000 до 300000    1330
от 300000 и более      1483
Name: debt, dtype: int64

Далее посчитаем количество клиентов, которые имели задолженность по кредитам с группировкой по категории доходов  и рассмотрим процентное соотношение должников в каждой категории.

In [27]:
# группируем и выводим результат
df_research.groupby('income_category')[['debt']].agg(['count', 'sum', 'mean'])

debt           
                    count  sum  mean
income_category                     
от 0 до 50000         372   23 6.18%
от 50000 до 100000   4091  331 8.09%
от 100000 до 150000  7160  624 8.72%
от 150000 до 200000  4764  405 8.50%
от 200000 до 250000  2254  164 7.28%
от 250000 до 300000  1330   88 6.62%
от 300000 и более    1483  106 7.15%

**Вывод**

Самыми дисциплинированными клиентами оказались люди, относящиеся к категории с наименьшим доходом до 50000, а также клиенты, уровень дохода которых составляет от 250000 до 300000. Наиболее высокий процент клиентов, имевших задолженность по возврату кредита, располагается в середине диапазона доходов: от 100000 до 200000. 

- Как разные цели кредита влияют на его возврат в срок?

Сначала сгруппируем клиентов по категориям займов.

In [28]:
# группируем и считаем
display(df_research.groupby('purpose_category')['debt'].count())

purpose_category
автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: debt, dtype: int64

Далее посчитаем количество клиентов, которые имели задолженность по кредитам с группировкой по категории займов и рассмотрим соотношение клиентов, имевших задолженность, по категориям.

In [29]:
# группируем и выводим результат
df_research.groupby('purpose_category')[['debt']].agg(['count', 'sum', 'mean'])

debt           
                  count  sum  mean
purpose_category                  
автомобиль         4306  403 9.36%
недвижимость      10811  782 7.23%
образование        4013  370 9.22%
свадьба            2324  186 8.00%

**Вывод**

Наименьший процент клиентов с просрочками находится среди тех, кто брал кредит для покупки или ремонта недвижимости, а самый высокий располагается среди кредитов на автомобиль и образование.

## Шаг 4. Общий вывод

Данные были предобработаны. Обнаружили и заменили пропуски в двух столбцах. Тот столбец, который не влиял на резултаты исследования, заполнили явными обозначениями, а необходимый нам столбец - медианными значениями. Привели текстовые данные к единому регистру и удалили явные дубликаты, хоть их количество было небольшим, привели числовые столбцы в необходимый тип данных. Сделали лемматизацию и категоризацию данных.

Были проанализированы некоторые зависимости и сделаны выводы по их проверке:

   1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
    
Зависимости между количеством количеством детей не обнаружено, но клиенты без детей наболее дисциплинированные.
   
   
   2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
   
Самые дисциплинированные клиенты оказались среди тех, кто является вдовой / вдовцом, а наименее дисциплинированные те, кто не состоит в официальном браке.
   
   
   3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   
Клиенты с наименьшим и наибольшим уровнем заработка имеют процент просрочек по кредиту меньше, чем клиенты со средним уровнем доходов.
   
   
   4. Как разные цели кредита влияют на его возврат в срок?
   
Клиенты, бравшие займ для покупки или ремонта недвижимости, являются наиболее дисциплинированными, а те, кто брал на образование и автомобиль - наоборот.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.